In [ ]:
import gradio as gr
#Defining base class ProfitReport
class ProfitReport:
  def __init__(self,month_name,total_sales,cost_of_goods_sold,labor_cost,occupancy_cost,operating_expenses):
    self.month_name=month_name
    self.__total_sales=total_sales
    self.cost_of_goods_sold=cost_of_goods_sold
    self.labor_cost=labor_cost
    self.occupancy_cost=occupancy_cost
    self.operating_expenses=operating_expenses

    # Calculate net profit during initialization
    self.net_profit_before = self._calculate_profit()

    # Creating the data dictionary as an instance attribute
    self.data_for_report = {
        'month': self.month_name,
        'revenue': self.__total_sales,
        'cogs': self.cost_of_goods_sold,
        'labor': self.labor_cost,
        'occupancy': self.occupancy_cost,
        'expenses': self.operating_expenses,
        'profit_before': self.net_profit_before
    }

  # Defining method for returning calculated profit
  def _calculate_profit(self):
    total_prime_cost = self.cost_of_goods_sold + self.labor_cost
    total_operating_cost = self.occupancy_cost + self.operating_expenses
    net_profit_before = self.__total_sales - total_prime_cost - total_operating_cost
    return net_profit_before

  #Defining method to generate report
  def generate_report(self):
    report_str = f'PROFIT REPORT FOR {self.month_name}'.center(120) + '\n'
    report_str += f"|{'Month':^10}|{'Revenue':^11}|{'Cost of goods sold':^18}|{'Labor Cost':^12}|{'Occupancy Cost':^16}|{'Operating Expenses':^20}|{'Net Profit':^12}\n"
    report_str += '-'*120 + '\n'
    report_str += f"|{self.data_for_report['month']:<10}|{self.data_for_report['revenue']:<11.2f}|{self.data_for_report['cogs']:<18}|{self.data_for_report['labor']:<12.2f}|{self.data_for_report['occupancy']:<16.2f}|{self.data_for_report['expenses']:<20.2f}|{self.data_for_report['profit_before']:<12.2f}\n"
    return report_str

#Creating child class TaxReport which inherits from parent class ProfitReport (Single Inheritance)
class TaxReport(ProfitReport):
  def __init__(self,month_name,tax_percentage,total_sales,cost_of_goods_sold,labor_cost,occupancy_cost,operating_expenses):
    super().__init__(month_name,total_sales,cost_of_goods_sold,labor_cost,occupancy_cost,operating_expenses)
    self.tax_percentage=tax_percentage

  #Defining method to calculate payable tax and return it along with net profit after tax
  def calc_tax(self):
    payable_tax = self.net_profit_before * (self.tax_percentage / 100)
    net_profit_after = self.net_profit_before - payable_tax
    return payable_tax,net_profit_after

  #Method for adding payable tax and net profit after tax to self.data_for_report dictionary
  def update_data_for_report(self):
    payable_tax,net_profit_after=self.calc_tax()
    self.data_for_report['tax']=payable_tax
    self.data_for_report['profit_after']=net_profit_after

  def generate_report(self):
    self.update_data_for_report()
    report_str = f'PROFIT & TAX REPORT FOR {self.month_name}'.center(155) + '\n'
    report_str += f"|{'Month':^10}|{'Revenue':^11}|{'Cost of goods sold':^18}|{'Labor Cost':^12}|{'Occupancy Cost':^16}|{'Operating Expenses':^20}|{'Net Profit before Tax':^23}|{'Payable Tax':^15}|{'Net Profit after Tax':^23}\n"
    report_str += '-'*155 + '\n'
    #Saving the data dictionary in a list
    report_list = [self.data_for_report]
    #Data traversing
    for i in report_list:
      report_str += f"|{i['month']:<10}|{i['revenue']:<11.2f}|{i['cogs']:<18}|{i['labor']:<12.2f}|{i['occupancy']:<16.2f}|{i['expenses']:<20.2f}|{i['profit_before']:<23.2f}|{i['tax']:<15.2f}|{i['profit_after']:<23.2f}\n"
    return report_str

#Defining a method outside of class to save and append saved data
def save_report(file_name,report_content):
  with open(file_name,'a') as file:
    file.write(report_content)
    print(f'Report saved to {file_name}')

# Defining gradio-compatible main function
def main(
    month_name: str,
    tax_percentage: float,
    total_sales: float,
    cost_of_goods_sold: float,
    labor_cost: float,
    occupancy_cost: float,
    operating_expenses: float
) -> str:

  month_name = month_name.upper()

  # Calculate net profit before tax
  view_net_profit_before = total_sales - cost_of_goods_sold - labor_cost - occupancy_cost - operating_expenses
  report_output_string = ""

  if view_net_profit_before <= 0:
    report_object = ProfitReport(month_name, total_sales, cost_of_goods_sold, labor_cost, occupancy_cost, operating_expenses)
    report_output_string = report_object.generate_report()
    report_output_string += "\n\nYou are not liable to pay taxes as your restaurant is facing losses"
  else:
    report_object = TaxReport(month_name, tax_percentage, total_sales, cost_of_goods_sold, labor_cost, occupancy_cost, operating_expenses)
    report_output_string = report_object.generate_report()

  return report_output_string

def save_report_gradio(file_name, report_content):
  if not file_name:
    return "Please enter a file name to save the report."
  try:
    # Use the existing save_report function from the above code
    save_report(file_name, report_content)
    return f"Report saved to {file_name}"
  except Exception as e:
    return f"Error saving report: {e}"

#Creating the gradio interface to accept inputs and display output
with gr.Blocks() as demo:
  gr.Markdown("## Profit and Tax Report Generator")
  with gr.Row():
    month_name = gr.Textbox(label='Month Name', placeholder='e.g., January')
    tax_percentage = gr.Number(label='Tax Percentage (%)', value=0.0)
  with gr.Row():
    total_sales = gr.Number(label='Total Sales', value=0.0)
    cost_of_goods_sold = gr.Number(label='Cost of Goods Sold', value=0.0)
  with gr.Row():
    labor_cost = gr.Number(label='Labor Cost', value=0.0)
    occupancy_cost = gr.Number(label='Occupancy Cost', value=0.0)
    operating_expenses = gr.Number(label='Other Operating Expenses', value=0.0)

  generate_report_btn = gr.Button('Generate Report')
  report_output_display = gr.Textbox(label="Generated Report", lines=15, interactive=False)

  with gr.Row():
    file_name_input = gr.Textbox(label='Filename to save (e.g., report.txt)', placeholder='report.txt')
    save_report_btn = gr.Button('Save Report')
  save_status_display = gr.Textbox(label="Save Status", interactive=False)

  #Buttons that will perform action on-click
  generate_report_btn.click(
      fn=main,
      inputs=[
          month_name, tax_percentage, total_sales, cost_of_goods_sold,
          labor_cost, occupancy_cost, operating_expenses
      ],
      outputs=report_output_display
  )

  save_report_btn.click(
      fn=save_report_gradio,
      inputs=[
          file_name_input, report_output_display
      ],
      outputs=save_status_display
  )

if __name__ == '__main__':
  demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://be430d195ffd8ccdb2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#Initial program without gradio
class ProfitReport:
  def __init__(self,month_name,total_sales,cost_of_goods_sold,labor_cost,occupancy_cost,operating_expenses):
    self.month_name=month_name
    self.__total_sales=total_sales
    self.cost_of_goods_sold=cost_of_goods_sold
    self.labor_cost=labor_cost
    self.occupancy_cost=occupancy_cost
    self.operating_expenses=operating_expenses

    # Calculate net profit and margin during initialization
    self.net_profit_before = self._calculate_profit()

    # Create the data dictionary as an instance attribute
    self.data_for_report = {
        'month': self.month_name,
        'revenue': self.__total_sales,
        'cogs': self.cost_of_goods_sold,
        'labor': self.labor_cost,
        'occupancy': self.occupancy_cost,
        'expenses': self.operating_expenses,
        'profit_before': self.net_profit_before
    }

  # Helper method for calculation, making it an instance method
  def _calculate_profit(self):
    total_prime_cost = self.cost_of_goods_sold + self.labor_cost
    total_operating_cost = self.occupancy_cost + self.operating_expenses
    net_profit_before = self.__total_sales - total_prime_cost - total_operating_cost
    return net_profit_before

  def generate_report(self):
    report_str = f'PROFIT REPORT FOR {self.month_name}'.center(120) + '\n'
    report_str += f"|{'Month':^10}|{'Revenue':^11}|{'Cost of goods sold':^18}|{'Labor Cost':^12}|{'Occupancy Cost':^16}|{'Operating Expenses':^20}|{'Net Profit':^12}\n"
    report_str += '-'*120 + '\n'
    report_str += f"|{self.data_for_report['month']:<10}|{self.data_for_report['revenue']:<11.2f}|{self.data_for_report['cogs']:<18}|{self.data_for_report['labor']:<12.2f}|{self.data_for_report['occupancy']:<16.2f}|{self.data_for_report['expenses']:<20.2f}|{self.data_for_report['profit_before']:<12.2f}\n"
    return report_str

class TaxReport(ProfitReport):
  def __init__(self,month_name,tax_percentage,*args,**kwargs):
    super().__init__(month_name,*args,**kwargs)
    self.tax_percentage=tax_percentage

  def calc_tax(self):
    payable_tax = self.net_profit_before * (self.tax_percentage / 100)
    net_profit_after = self.net_profit_before - payable_tax
    return payable_tax,net_profit_after

  def update_data_for_report(self):
    payable_tax,net_profit_after=self.calc_tax()
    self.data_for_report['tax']=payable_tax
    self.data_for_report['profit_after']=net_profit_after

  def generate_report(self):
    self.update_data_for_report()
    report_str = f'PROFIT & TAX REPORT FOR {self.month_name}'.center(155) + '\n'
    report_str += f"|{'Month':^10}|{'Revenue':^11}|{'Cost of goods sold':^18}|{'Labor Cost':^12}|{'Occupancy Cost':^16}|{'Operating Expenses':^20}|{'Net Profit before Tax':^23}|{'Payable Tax':^15}|{'Net Profit after Tax':^23}\n"
    report_str += '-'*155 + '\n'
    report_list = [self.data_for_report]
    for i in report_list:
      report_str += f"|{i['month']:<10}|{i['revenue']:<11.2f}|{i['cogs']:<18}|{i['labor']:<12.2f}|{i['occupancy']:<16.2f}|{i['expenses']:<20.2f}|{i['profit_before']:<23.2f}|{i['tax']:<15.2f}|{i['profit_after']:<23.2f}\n"
    return report_str

def save_report(file_name,report_content):
  with open(file_name,'a') as file:
    file.write(report_content)
    print(f'Report saved to {file_name}')

def main():
  while True:
    try:
      month_name=input("Enter month name:").upper()
      tax_percentage=float(input('Enter tax percentage(%):'))
      total_sales=float(input(f'Enter total sales:'))
      cost_of_goods_sold=float(input(f'Enter cost of goods sold:'))
      labor_cost=float(input('Enter labor cost:'))
      occupancy_cost=float(input('Enter occupancy cost:'))
      operating_expenses=float(input('Enter other operating costs:'))
    except ValueError:
      print('Invalid input. Please enter valid numbers.')
      continue

    # Calculate view_net_profit_before here, after successful input collection
    view_net_profit_before=total_sales-cost_of_goods_sold-labor_cost-occupancy_cost-operating_expenses

    report_object = None # Initialize to None
    report_output_string = "" # Initialize an empty string for the report content

    if view_net_profit_before <= 0:
      report_object=ProfitReport(month_name,total_sales,cost_of_goods_sold,labor_cost,occupancy_cost,operating_expenses)
      report_output_string = report_object.generate_report()
      print(report_output_string)
      print("You are not liable to pay taxes as your restaurant is facing losses")
    else:
      report_object=TaxReport(month_name,tax_percentage,total_sales,cost_of_goods_sold,labor_cost,occupancy_cost,operating_expenses)
      report_output_string = report_object.generate_report()
      print(report_output_string)

    save_data=input('Do you want to save this report? (yes/no): ').lower()
    if save_data == 'yes':
      file_name=input("Enter file name to save the report (eg: filename.txt): ")
      save_report(file_name,report_output_string)

    another_report = input('Do you want to generate another report? (yes/no): ').lower()
    if another_report != 'yes':
      break

if __name__ == '__main__':
  main()

Enter month name:February
Enter tax percentage(%):15
Enter total sales:100000
Enter cost of goods sold:32000
Enter labor cost:28000
Enter occupancy cost:8000
Enter other operating costs:12000
                                                              PROFIT & TAX REPORT FOR FEBRUARY                                                             
|  Month   |  Revenue  |Cost of goods sold| Labor Cost | Occupancy Cost | Operating Expenses | Net Profit before Tax |  Payable Tax  | Net Profit after Tax  
-----------------------------------------------------------------------------------------------------------------------------------------------------------
|FEBRUARY  |100000.00  |32000.0           |28000.00    |8000.00         |12000.00            |20000.00               |3000.00        |17000.00               

Do you want to save this report? (yes/no): yes
Enter file name to save the report (eg: filename.txt): february_report.txt
Report saved to february_report.txt
Do you want to genera

In [ ]:
#Few test cases for the program:

#1.Entering invalid input
total_sales=float(input(f'Enter total sales:'))
#User input: Ten Thousand
"""
Expected output: Invalid input. Please enter valid numbers.
"""

#2.Generation of Profit Report only
profit_report=ProfitReport('January',1000,500,200,300,100)
print(profit_report.generate_report())
"""
Expected output: Report containing only 'Net Profit' (negative value) with a message saying:
 'You are not liable to pay taxes as your restaurant is facing losses'
"""

#3.Generation of Tax amount with Profit Report
tax_report=TaxReport('January',10,1000,500,200,300,100)
print(tax_report.generate_report())
"""
Expected output: Report containing 'Net Profit before Tax', 'Payable Tax', and 'Net Profit after Tax'
"""

#4.Saving data
save_data=input('Do you want to save this report? (yes/no): ').lower()
#User input : yes
if save_data == 'yes':
  file_name=input("Enter file name to save the report (eg: filename.txt): ")
#User input : savereport.txt
"""
Expected output: Report saved to savereport.txt
"""